# Centralities Calculation

In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), "../../"))

sys.path.append(parent_dir)

from assignment_1.lib.graph import Graph
from assignment_1.lib.load_romanian_cities import load_romania

romania_map = load_romania()

### Degree Centrality

In [7]:
def degreeCentrality(cities):
    degreeCentralityTable = []
    for city in cities.get_nodes():
        degreeCentralityTable.append((city,len(cities[city])))

    degreeCentralityTable.sort(key=lambda x:x[1],reverse=True)
    return degreeCentralityTable

print(degreeCentrality(romania_map))

[('Oradea', 2), ('Zerind', 2), ('Arad', 2), ('Timisoara', 2), ('Lugoj', 2), ('Mehadia', 2), ('Drobeta', 2), ('Craiova', 2), ('Sibiu', 2), ('Rimnicu Vilcea', 2), ('Fagaras', 2), ('Pitesti', 2), ('Giurgiu', 2), ('Bucharest', 2), ('Urziceni', 2), ('Eforie', 2), ('Hirsova', 2), ('Vaslui', 2), ('Iasi', 2), ('Neamt', 2)]


### Closeness Centrality

In [2]:
#used dijkstras algorithm to find the closest possible distance between the two cities.this is because my dfs agorithm doesn't nessesarly return the
#shortest distance
def dijkstra(graph: Graph, start: str) -> list[float]:
    distances = {node: float('inf') for node in graph.get_nodes()}
    distances[start] = 0
    
    queue = [start]
    while queue:
        current_node = queue.pop(0)
        for neighbor in graph[current_node][1]:
            tentative_distance = distances[current_node] + 1
            if tentative_distance < distances[neighbor]:
                distances[neighbor] = tentative_distance
                queue.append(neighbor)
    
    return distances

def closeness_centrality(graph: Graph):
    closeness = {}
    for node in graph.get_nodes():
        distances = dijkstra(graph, node)
        total_distance = sum(distances.values())
        closeness[node] = 1 / total_distance
    
    return closeness

romania_closeness = closeness_centrality(romania_map)
print(romania_closeness)

{'Oradea': 0.014285714285714285, 'Zerind': 0.0125, 'Arad': 0.015625, 'Timisoara': 0.013157894736842105, 'Lugoj': 0.011363636363636364, 'Mehadia': 0.011111111111111112, 'Drobeta': 0.0125, 'Craiova': 0.014705882352941176, 'Sibiu': 0.018518518518518517, 'Rimnicu Vilcea': 0.017857142857142856, 'Fagaras': 0.01818181818181818, 'Pitesti': 0.017543859649122806, 'Giurgiu': 0.014084507042253521, 'Bucharest': 0.018867924528301886, 'Urziceni': 0.01639344262295082, 'Eforie': 0.010526315789473684, 'Hirsova': 0.012987012987012988, 'Vaslui': 0.013333333333333334, 'Iasi': 0.01098901098901099, 'Neamt': 0.009174311926605505}


### Katz Centrality

In [3]:
import numpy as np

def katz_centrality(graph, alpha, beta):
    n = len(graph.get_nodes())
    A = np.zeros((n,n))
    nodes = list(graph.get_nodes())
    node_dict = {nodes[i]:i for i in range(n)}
    
    # Create adjacency matrix
    for node in nodes:
        neighbors = graph[node][1]
        i = node_dict[node]
        for neighbor in neighbors:
            j = node_dict[neighbor]
            A[i][j] = 1
    
    # Calculate Katz centrality
    I = np.identity(n)
    M = beta * np.linalg.inv(I - alpha * A)
    katz = np.sum(M, axis=1)
    
    katz_dict = {nodes[i]:katz[i] for i in range(n)}
    return katz_dict

katz = katz_centrality(romania_map, 0.1, 0.1)
print(katz)

{'Oradea': 0.12809235103167865, 'Zerind': 0.1268843841077829, 'Arad': 0.1407514900461499, 'Timisoara': 0.1265913901447122, 'Lugoj': 0.12516241140097198, 'Mehadia': 0.125032723865008, 'Drobeta': 0.12516482724910785, 'Craiova': 0.1266155486260703, 'Sibiu': 0.15403912620900395, 'Rimnicu Vilcea': 0.1409906590115949, 'Fagaras': 0.13055676200061583, 'Pitesti': 0.12925191528087498, 'Giurgiu': 0.11515284937971552, 'Bucharest': 0.15152849379715486, 'Urziceni': 0.1403234113103422, 'Eforie': 0.11252851930616509, 'Hirsova': 0.12528519306165073, 'Vaslui': 0.12642042624461622, 'Iasi': 0.12388085113581981, 'Neamt': 0.112388085113582}


### Eigen-vector Centrality

In [4]:
import numpy as np

def eigenvector_centrality(graph):
    adjacency = np.zeros((len(graph.get_nodes()), len(graph.get_nodes())))
    node_index = {node: i for i, node in enumerate(graph.get_nodes())}
    for node in graph.get_nodes():
        neighbors = graph[node][1]
        for neighbor in neighbors:
            adjacency[node_index[node], node_index[neighbor]] = 1
    
    centrality = np.ones(len(graph.get_nodes()))
    for i in range(100):
        centrality = np.matmul(adjacency, centrality)
        centrality = centrality / np.linalg.norm(centrality)
    
    return {node: centrality[node_index[node]] for node in graph.get_nodes()}

romania_eigenvector = eigenvector_centrality(romania_map)
print(romania_closeness)

{'Oradea': 0.014285714285714285, 'Zerind': 0.0125, 'Arad': 0.015625, 'Timisoara': 0.013157894736842105, 'Lugoj': 0.011363636363636364, 'Mehadia': 0.011111111111111112, 'Drobeta': 0.0125, 'Craiova': 0.014705882352941176, 'Sibiu': 0.018518518518518517, 'Rimnicu Vilcea': 0.017857142857142856, 'Fagaras': 0.01818181818181818, 'Pitesti': 0.017543859649122806, 'Giurgiu': 0.014084507042253521, 'Bucharest': 0.018867924528301886, 'Urziceni': 0.01639344262295082, 'Eforie': 0.010526315789473684, 'Hirsova': 0.012987012987012988, 'Vaslui': 0.013333333333333334, 'Iasi': 0.01098901098901099, 'Neamt': 0.009174311926605505}


### Between-ness Centrality

In [5]:
def betweenness_centrality(graph):
    betweenness = {node: 0 for node in graph.get_nodes()}

    for node in graph.get_nodes():
        S = []
        P = {v: [] for v in graph.get_nodes()}
        sigma = {v: 0 for v in graph.get_nodes()}
        sigma[node] = 1
        D = {v: -1 for v in graph.get_nodes()}
        D[node] = 0

        Q = [node]
        while Q:
            v = Q.pop(0)
            S.append(v)

            for w in graph[v][1]:
                if D[w] < 0:
                    Q.append(w)
                    D[w] = D[v] + 1
                if D[w] == D[v] + 1:
                    sigma[w] += sigma[v]
                    P[w].append(v)

        delta = {v: 0 for v in graph.get_nodes()}
        while S:
            w = S.pop()
            for v in P[w]:
                delta[v] += (sigma[v] / sigma[w]) * (1 + delta[w])
            if w != node:
                betweenness[w] += delta[w]

    return betweenness
print(betweenness_centrality(romania_map))

{'Oradea': 13.666666666666666, 'Zerind': 3.6666666666666665, 'Arad': 74.66666666666666, 'Timisoara': 37.0, 'Lugoj': 14.0, 'Mehadia': 10.666666666666666, 'Drobeta': 29.666666666666668, 'Craiova': 53.0, 'Sibiu': 131.66666666666669, 'Rimnicu Vilcea': 88.0, 'Fagaras': 96.0, 'Pitesti': 64.0, 'Giurgiu': 0, 'Bucharest': 182.0, 'Urziceni': 152.0, 'Eforie': 0, 'Hirsova': 36.0, 'Vaslui': 68.0, 'Iasi': 36.0, 'Neamt': 0}
